<a href="https://colab.research.google.com/github/Hee0305/DL/blob/main/FT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning and Visualization (CNN's)
- cifar-10
- resnet50
- mobilenetV2

> https://medium.com/@andrew.dabydeen/transfer-learning-using-resnet50-and-cifar-10-6242ed4b4245

### Transfer learning (imagenet)
- ImageNet dataset 
    - 1.4 million labeled images 
    - 1,000 different classes  
    
tensorflow.keras.applications
- Xception
- Inception V3
- ResNet50
- VGG16
- VGG19
- MobileNet

> [ref1-architecture-comparison-of-alexnet-vggnet-resnet-inception-densenet](https://towardsdatascience.com/architecture-comparison-of-alexnet-vggnet-resnet-inception-densenet-beb8b116866d)

> [ref2-imagenet-vggnet-resnet-inception-xception-keras](https://www.pyimagesearch.com/2017/03/20/imagenet-vggnet-resnet-inception-xception-keras/)

This Colab notebook demonstrates transfer learning with a pretrained ConvNet: MobileNetV2

0.   Data pipeline
1.   Baseline model: train a simple CNN from scratch
2.   Transfer learning: pretraiend ConvNet as a feature extractor
3.   Transfer learning: fine-tune a pretrained ConvNet
4.   Test accuracy & visualize predictions



Before beginning, let's load the appropriate libraries needed for this notebook

# ConvNet: MobileNetV2

In [70]:
# from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import  MobileNetV2, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import tensorflow as tf
from keras.utils import np_utils
from keras.models import load_model
from keras.datasets import cifar10
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
# import cv2
print(tf.__version__)

2.7.0


In [71]:
tf.keras.backend.clear_session()  # For memory

# GPU check
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print("#### ", len(gpus), "Physical GPUs,",
                 len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print('#### No CUDA supported GPU in this computer.')

####  1 Physical GPUs, 1 Logical GPUs


## Load data 
- Use total data

In [72]:
from keras.datasets import cifar10
# load dataset
(trainX, trainy), (testX, testy) = cifar10.load_data()
# summarize loaded dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))

#training의 10% split -> numpy 코드 이용(사이킷런)

Train: X=(50000, 32, 32, 3), y=(50000, 1)
Test: X=(10000, 32, 32, 3), y=(10000, 1)


cifar 10% -- 5000/1000

In [73]:


from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = \
            train_test_split(trainX,trainy,
                             random_state=42,
                             test_size=0.1,
                             stratify=trainy)

Xtrain = X_test
print(Xtrain.shape) # X_train
ytrain = y_test
print(ytrain.shape) # y_train


X_train,X_test,y_train,y_test = \
            train_test_split(testX,testy,
                             random_state=42,
                             test_size=0.1,
                             stratify=testy)

print(X_test.shape) # X_test
print(y_test.shape) # y_test

(5000, 32, 32, 3)
(5000, 1)
(1000, 32, 32, 3)
(1000, 1)


## *Preprocess the dataset*

In [ ]:
def preprocess_image_input(input_images): #이미지 픽셀 값 정수 -> 실수, preprocess_input함수로 바꿔줌(정규화)
  input_images = input_images.astype('float32')
  output_ims = preprocess_input(input_images)
  return output_ims

In [ ]:
#전처리 된 데이터
x_train = preprocess_image_input(Xtrain)
x_test = preprocess_image_input(X_test)

In [ ]:
x_train.shape ,X_test.shape #,len(x_train),x_train[4].shape

In [ ]:
y_train=ytrain
y_test=y_test
y_train.shape ,y_test.shape #레이블 구조: 2차원 행벡터

augmentation

#### Model Creation

Next, let's load MobileNetV2 with just the convolutional layers and not the dense layers so we can train our new dataset on the new dense layers that we create

In [ ]:
TL_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) #imagenet: 1000개의 클래스, 140만개의 이미지


In [ ]:
len(TL_base.layers)

In [ ]:
# names of TL_base
print([x.name for x in TL_base.layers],end=' ') #layer 이름 출력

### Check the start of trainable layer

In [ ]:
TL_base.layers[81].name, TL_base.layers[107].name # mobilenetV2


Let's get an idea on how the ResNet architecture looks

In [ ]:
TL_base.summary()

In [ ]:
TL_base.layers[-1].get_config() # (7, 7, 1280), 맨 마지막 pooling 통과한 결과, 맨 마지막 index의 구조

For the new dataset, let's work with the cifar10 dataset which we can load directly from keras' dataset library. The Cifar10 data description is as follows - "The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images"

## Sequential model
- TL base: resnet

In [ ]:
# Set TL_base trainable or not.
TL_base.trainable = False  # Use the representative features pretrained by resnet.

In [ ]:
model = models.Sequential()
model.add(keras.Input(shape=(32,32,3))) #image: 224*224*3
model.add(layers.UpSampling2D((7,7)))  # 32*7 = 224 #가로, 세로 각각 7배씩 => 크기 확대
model.add(TL_base) #input을 TL_base에 넣어줌, frozen - convolution단계는 고정(기존 파라미터 이용)
model.add(layers.GlobalAveragePooling2D()) # (7,7,2048) => (2048,) #1280개짜리 벡터 만들어짐
model.add(layers.Flatten()) #fcn 설계
model.add(layers.BatchNormalization())
model.add(layers.Dense(1024, activation='relu')) #fcn1
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(512, activation='relu')) #fcn2
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(10, activation='softmax')) #output layer
#fcn 3층 설계

# model.compile(optimizer=optimizers.RMSprop(lr=2e-5), loss='binary_crossentropy', metrics=['acc'])
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 
model.compile(optimizer='SGD', 
                loss='sparse_categorical_crossentropy',  # sparse_categorical_crossentropy
                metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Plot model
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True, show_layer_names=True, to_file='model_S.png')

In [ ]:
from tensorflow.keras import layers, models, callbacks

In [ ]:
#여기부터 중요
#content에 model 폴더 생성 - 높아질때마다 model폴더에 생성
mc_callback = callbacks.ModelCheckpoint(filepath="./model/cifar10_pct100_mobilenetV2S_SGD_best_weights.{epoch:03d}-{val_accuracy:.4f}.hdf5", 
                              monitor='val_accuracy', verbose=0, save_best_only=True)
es_callback = callbacks.EarlyStopping(monitor='val_accuracy', 
                            mode='max', verbose=1, patience=5)

In [ ]:
%%time
history = model.fit(x_train, y_train, #5만장
                    epochs=100, batch_size=50, 
                    validation_data=(x_test, y_test), #만장
                    callbacks=[mc_callback,es_callback])

## Evaluate the Model

Calculate the loss and accuracy metrics using the model's `.evaluate` function.

In [ ]:
model.evaluate(x_test, y_test, batch_size=50)

## 훈련 데이터와 검증 데이터에 대한 loss, accuracy 시각화

In [ ]:
# 훈련 데이터와 검증 데이터에 대한 loss 시각화.
epochs = range(1, len(history.history['loss']) + 1)

loss_list = history.history['loss'] #[100 * i for i in history.history['loss']]
vloss_list = history.history['val_loss'] #[100 * i for i in history.history['val_loss']]

plt.plot(epochs,loss_list)  
plt.plot(epochs,vloss_list)

plt.plot(np.argmin(np.array(vloss_list))+1,vloss_list[np.argmin(np.array(vloss_list))], 'r*')
plt.title('cifar10_100%: TL(mobilenetV2) Sequential model - val_loss, min:' + str(np.round(vloss_list[np.argmin(np.array(vloss_list))],3)))
plt.ylabel('val-Loss (%)')
plt.xlabel('Epoch')
plt.legend(['loss','val_loss','best'], loc='best')
plt.show()

In [ ]:
# 훈련 데이터와 검증 데이터에 대한 accuracy 시각화.
epochs = range(1, len(history.history['accuracy']) + 1)

acc_list = [100 * i for i in history.history['accuracy']]
vacc_list = [100 * i for i in history.history['val_accuracy']]

plt.plot(epochs,acc_list)  
plt.plot(epochs,vacc_list)

plt.plot(np.argmax(np.array(vacc_list))+1,vacc_list[np.argmax(np.array(vacc_list))], 'r*')
plt.title('cifar10_100%: TL(mobilenetV2) Sequential model - val_accuracy, max:' + str(np.round(vacc_list[np.argmax(np.array(vacc_list))],3)))
plt.ylabel('val-Accuracy (%)')
plt.xlabel('Epoch')
plt.legend(['accuracy','val_accuracy','best'], loc='best')
plt.show()

In [ ]:
# model.save('cifar10_SGD_ep20acc88.hdf5')
# # Load the saved model
# model = tf.keras.models.load_model('cifar10_SGD_ep20acc88.hdf5')
# model.evaluate(x_test, y_test)

### Visualization loss & accuracy

The training/validation loss and accuracy visualizations are shown below

In [ ]:
# More graphs of loss and accuracy
history_dict = history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(loss) + 1)

plt.figure(figsize=(14, 4))

plt.subplot(1,2,1)
plt.plot(epochs, loss, 'go-', label='Training Loss')
plt.plot(epochs, val_loss, 'bd', label='Validation Loss')
plt.plot(np.argmin(np.array(val_loss))+1,val_loss[np.argmin(np.array(val_loss))], 'r*', ms=12)
plt.title('Training and Validation Loss, min: ' + str(np.round(val_loss[np.argmin(np.array(val_loss))],3)))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

epochs = range(1, len(loss) + 1)

plt.subplot(1,2,2)
plt.plot(epochs, acc, 'go-', label='Training Accuracy') #, c='blue')
plt.plot(epochs, val_acc, 'bd', label='Validation Accuracy') #, c='red')
plt.plot(np.argmax(np.array(val_acc))+1,val_acc[np.argmax(np.array(val_acc))], 'r*', ms=12)
plt.title('Training and Validation Accuracy, max: ' + str(np.round(val_acc[np.argmax(np.array(val_acc))],3)))
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

#fcn만 설계

### Fine tuning

In [ ]:
# Unfreeze all layers in base model
tf.keras.backend.clear_session()  # For memory
TL_base.trainable = True #TL_base:convolution 단계 -> 훈련 대상으로 변경

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(TL_base.layers))

In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = 107  # 81, 107 for mobilenetV2

# Freeze all the layers before the `fine_tune_at` layer #내가 훈련시키는 위치 설정 기준 
for layer in TL_base.layers[:fine_tune_at]:
  layer.trainable =  False #107 이전 파라미터는 frozen(고정)

#convnet&fcn 그림 보기(pdf)

In [ ]:
# Compile model - model: epoch 42까지 훈련시킨 모델 -> 107이후부터는 내가 훈련시킨 모델로(original과의 차이점)
# 주의
model.compile(optimizer='SGD', #최적화
                loss='sparse_categorical_crossentropy',  # sparse_categorical_crossentropy - 시간 단축
                metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
mc_callback = callbacks.ModelCheckpoint(filepath="./model/cifar10_pct100_mobileNetV2S_FT_SGD_best_weights.{epoch:03d}-{val_accuracy:.4f}.hdf5", 
                              monitor='val_accuracy', verbose=0, save_best_only=True)
es_callback = callbacks.EarlyStopping(monitor='val_accuracy', 
                            mode='max', verbose=1, patience=5)

## Train the fine-tuned model

In [ ]:
INITIAL_EPOCHS = len(loss) #42개(epoch)
FINE_TUNE_EPOCHS = 100
TOTAL_EPOCHS = INITIAL_EPOCHS + FINE_TUNE_EPOCHS #43부터 학습 시작

In [ ]:
%%time
history_fine = model.fit(x_train, y_train, 
                    epochs=TOTAL_EPOCHS, 
                    initial_epoch=INITIAL_EPOCHS, #앞의 모델에서 42번까지 학습, 앞에서 한 42번 학습까지는 그대로 유지 그 이후부터 학습
                    batch_size=50, 
                    validation_data=(x_test, y_test),
                    callbacks=[mc_callback,es_callback])

In [ ]:
model.evaluate(x_test, y_test, batch_size=50)

## Display training curve

In [ ]:
# Display training curve

acc = history.history['accuracy'] + history_fine.history['accuracy'] #튜닝 전+후
val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']

plt.figure(figsize=(8, 4))

plt.plot(acc, label='Train accuracy')
plt.plot(val_acc, label='Val accuracy')
# plt.ylim([0.8, 1])
plt.plot([INITIAL_EPOCHS-1, INITIAL_EPOCHS-1], plt.ylim(ymin=acc[0]), label='Start Fine Tuning') #41-직선, 42부터 fine tuning 시작
plt.title("Fine-tune a Pretrained ConvNet:MobileNetV2-S")
plt.legend(loc='upper left')

plt.show()

In [ ]:
# More graphs of loss and accuracy in Fine Tuning - 43부터 그림, 90%로 정확도 높아짐
history_dict = history_fine.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(INITIAL_EPOCHS, INITIAL_EPOCHS+len(loss))

plt.figure(figsize=(14, 4))

plt.subplot(1,2,1)
plt.plot(epochs, loss, 'go-', label='Training Loss')
plt.plot(epochs, val_loss, 'bd', label='Validation Loss')
plt.plot(INITIAL_EPOCHS + np.argmin(np.array(val_loss))+1,val_loss[np.argmin(np.array(val_loss))], 'r*', ms=12)
plt.title('Training and Validation Loss, min: ' + str(np.round(val_loss[np.argmin(np.array(val_loss))],3)))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

# epochs = range(1, len(loss_values) + 1)

plt.subplot(1,2,2)
plt.plot(epochs, acc, 'go-', label='Training Accuracy')#, c='blue')
plt.plot(epochs, val_acc, 'bd', label='Validation Accuracy') #, c='red')
plt.plot(INITIAL_EPOCHS + np.argmax(np.array(val_acc))+1,val_acc[np.argmax(np.array(val_acc))], 'r*', ms=12)
plt.title('Fine Tuning: Training and Validation Accuracy, max: ' + str(np.round(val_acc[np.argmax(np.array(val_acc))],3)))
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

### Visualize predictions
We can take a look at the predictions on the validation set.

In [ ]:
#최종 모델로 테스트 데이터에서 예측
probabilities = model.predict(x_test, batch_size=50)
probabilities = np.argmax(probabilities, axis = 1)

display_images(testX, probabilities, testy, "Predictions of the validation data")

---

In [ ]:
tf.keras.backend.clear_session()  # For memory

---